# Aluno: Guilherme Cristiano Goll

Exercício de classificação de texto, com aplicação de técnicas para normalização (tokenização, bag of words, stematização, etc).

Ao final é apresentada a conclusão sobre os resultados obtidos após a execução do classificador com Regressão Logística e o SIA (Sentiment Intensity Analyzer) do NLKT.

Observação: parte da limpeza do dataset (csv) foi realizada através de ferramenta externa, para facilitar a importação do arquivo no notebook.

In [1]:
import os
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('.\\imdb.csv', sep='|')

In [3]:
#df.head()
df[df.sentiment.isnull()]

,review,sentiment


In [4]:
df.sentiment.value_counts()
df[df.review == ''].count()

review       0
sentiment    0
dtype: int64

In [21]:
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guilh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\guilh\AppData\Roaming\nltk_data...


True

In [6]:
def normalize_df(df):
    '''Normalize dataframe, applying tokenization and other techniques
    '''
    ps = PorterStemmer()
    
    stop_words=set(stopwords.words("english"))
    
    def normalize(text: str):        
        final_sent = []
        tokenized_sent = sent_tokenize(text.strip())
        for sent in tokenized_sent:
            tokenized_words = word_tokenize(sent)
            for word in tokenized_words:
                if word not in stop_words:
                    stemmed_word = ps.stem(word)
                    final_sent.append(' ' + stemmed_word)
        
        return ''.join(final_sent)
    
    for idx in range(len(df)):
        df.iat[idx, 0] = normalize(df.iloc[idx, 0])
    
    return df

In [7]:
df = normalize_df(df)
df.head()

,review,sentiment
0,one review mention watch 1 Oz episod 'll hook...,positive
1,A wonder littl product . the film techniqu un...,positive
2,I thought wonder way spend time hot summer we...,positive
3,basic 's famili littl boy ( jake ) think 's z...,negative
4,petter mattei 's love time money visual stun ...,positive


# Classificador com Regressão Logística

In [8]:
#Divisão das variáveis preditoras e variáveis preditas
X = df['review']
y = df['sentiment']

In [9]:
#Criação do bag of words
vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
X = vectorizer.fit_transform(X)

In [11]:
#Divisão do dataset entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=42)

In [12]:
y_train = y_train.ravel()

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1,
                           solver='saga',
                           penalty='elasticnet',
                           l1_ratio=0.2,
                           class_weight='balanced',
                           max_iter=6000,
                           dual=False)
model = model.fit(X_train, y_train)

In [13]:
#Apuração da acurácia do modelo
model.score(X_test, y_test)

0.6584

# Classificador Naive Bayes

In [14]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(df['review'])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, df['sentiment'], test_size=0.3, random_state=42)

In [16]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)
clf.score(X_test, y_test)

0.8498

# Predição com SIA

In [33]:
from nltk.sentiment import SentimentIntensityAnalyzer

X_train, X_test, y_train, y_test = train_test_split(
    df[['review']], df[['sentiment']], stratify=df['sentiment'], test_size=0.3, random_state=42)

In [61]:
def sia_predict(sia: SentimentIntensityAnalyzer, tx: str) -> int:
    return 'positive' if sia.polarity_scores(tx)["compound"] > 0 else 'negative'

sia_y_pred = []

sia = SentimentIntensityAnalyzer()

for i, tx in enumerate(X_test.review):
    sia_y_pred.append(sia_predict(sia, tx))

In [82]:
print(sia_y_pred[6] == y_test.iloc[6][0])

True


In [94]:
verdadeiro_positivo = 0
verdadeiro_negativo = 0
for i in range(len(y_test)):
    answer = y_test.iloc[i][0]
    if answer == sia_y_pred[i]:
        if answer == 'positive':
            verdadeiro_positivo += 1
        elif answer == 'negative':
            verdadeiro_negativo += 1

In [95]:
total_positivo = y_test[y_test == 'positive'].count()[0]
total_negativo = y_test[y_test == 'negative'].count()[0]
print(f'Total de verdadeiros positivos: {verdadeiro_positivo}/{total_positivo} = {(verdadeiro_positivo/total_positivo) * 100 :.2f}%')
print(f'Total de verdadeiros negativos: {verdadeiro_negativo}/{total_negativo} = {(verdadeiro_negativo/total_negativo) * 100 :.2f}%')
print(f'Acurárcia geral: {(verdadeiro_positivo + verdadeiro_negativo) / len(y_test) * 100 :.2f}%')

Total de verdadeiros positivos: 6215/7500 = 82.87%
Total de verdadeiros negativos: 3575/7500 = 47.67%
Acurárcia geral: 65.27%


# Conclusão

A execução da Regressão Logística x SIA retornaram resultados bastante semelhantes em termos de acurácia: 65,84%, contra 65,27%, respectivamente. Porém, vale ressaltar que não foram exploradas todas as técnicas de pré-processamento dos dados, como Lematização, por exemplo. Neste sentido, acredito que ao trabalhar de uma forma mais profunda na etapa de normalização, deve ser possível obter melhores resultados.

Por curiosidade, foi aplicado ainda a classificação com o algoritmo Naive Bayes Multinomial, que apresentou um resultado consideravalmente superior em termos de acurácia: 84,98%.